In [1]:
# importing all necessary libraries

import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from nilearn import plotting
import os

In [2]:
# creating a function which will take raw data, mask data, new file path

def update_header_and_save(raw_filepath, mask_filepath, new_mask_filepath):
    raw_img = nib.load(raw_filepath)
    mask_img = nib.load(mask_filepath)

    # values to be updated
    attributes = ['extents', 'dim_info', 'slice_end', 'cal_max', 'cal_min', 
                  'glmax', 'glmin', 'srow_x', 'srow_y', 'srow_z']

    # copying header and updating each val in attributes according to raw_img
    header = mask_img.header.copy()
    for attr in attributes:
        header[attr] = raw_img.header[attr]

    # create new vox
    new_affine = np.eye(4)
    new_affine[:-1, :] = np.array([header['srow_x'], header['srow_y'], header['srow_z']])

    # saving an img to new folder
    updated_mask_img = nib.Nifti1Image(mask_img.get_fdata(), new_affine, header)
    nib.save(updated_mask_img, new_mask_filepath)

In [3]:
# allocating current directory
!pwd

/home/sharapova/test/notebooks


In [41]:
data_dir = '/home/sharapova/test/data/processed_with_newweight_3006/'
data_dir

'/home/sharapova/test/data/processed_with_newweight_3006/'

In [42]:
raw_dir = os.path.join(data_dir, 'raw_data')
print(raw_dir)
mask_dir = os.path.join(data_dir, 'mask')
print(mask_dir)
new_mask_dir = os.path.join(data_dir, 'new_mask_noise')
print(new_mask_dir)

/home/sharapova/test/data/processed_with_newweight_3006/raw_data
/home/sharapova/test/data/processed_with_newweight_3006/mask
/home/sharapova/test/data/processed_with_newweight_3006/new_mask_noise


In [43]:
raw_files = os.listdir(raw_dir)
len(raw_files)

52

In [44]:
for raw_file in raw_files:
    raw_filepath = os.path.join(raw_dir, raw_file)
    mask_filepath = os.path.join(mask_dir, raw_file)
    new_mask_filepath = os.path.join(new_mask_dir, os.path.splitext(raw_file)[0] + '_mask.nii')
    update_header_and_save(raw_filepath, mask_filepath, new_mask_filepath)

# Testing new function which will removing outside noise

In [34]:
from scipy.ndimage import generate_binary_structure
from skimage.morphology import label, opening
from skimage.measure import regionprops
import numpy as np
import nibabel as nib
import cv2 as cv

def clean_mask_advanced(mask, volume_threshold=500, opening_size=0.9):
    selem = generate_binary_structure(3, 2)
    mask_opened = opening(mask, selem)

    labeled_mask = label(mask_opened)

    # Identify regions above the volume threshold
    regions = regionprops(labeled_mask)
    large_regions = [r for r in regions if r.area > volume_threshold]

    # Create a clean mask
    cleaned_mask = np.zeros_like(mask)
    for region in large_regions:
        cleaned_mask[labeled_mask == region.label] = 1

    return cleaned_mask

def update_header_and_save(raw_filepath, mask_filepath, new_mask_filepath):
    raw_img = nib.load(raw_filepath)
    mask_img = nib.load(mask_filepath)

    mask_data = mask_img.get_fdata()
    cleaned_mask = clean_mask_advanced(mask_data)

    for i in range(cleaned_mask.shape[2]):

        slice_img = cleaned_mask[:, :, i]
    
        slice_img_8bit = cv.normalize(slice_img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')

        kernel = np.ones((5, 5), np.uint8)
        closing = cv.morphologyEx(slice_img_8bit, cv.MORPH_CLOSE, kernel)

        cleaned_mask[:, :, i] = closing

    # values to update
    attributes = ['extents', 'dim_info', 'slice_end', 'cal_max', 'cal_min', 
                  'glmax', 'glmin', 'srow_x', 'srow_y', 'srow_z']

    header = mask_img.header.copy()
    for attr in attributes:
        header[attr] = raw_img.header[attr]

    new_affine = np.eye(4)
    new_affine[:-1, :] = np.array([header['srow_x'], header['srow_y'], header['srow_z']])

    # saving an img to new folder
    updated_mask_img = nib.Nifti1Image(cleaned_mask, new_affine, header)
    nib.save(updated_mask_img, new_mask_filepath)

In [40]:
for raw_file in raw_files:
    raw_filepath = os.path.join(raw_dir, raw_file)
    mask_filepath = os.path.join(mask_dir, raw_file)
    new_mask_filepath = os.path.join(new_mask_dir, os.path.splitext(raw_file)[0] + '_mask.nii')
    update_header_and_save(raw_filepath, mask_filepath, new_mask_filepath)